这是一个多分类（multilabel, multiclass classification）的问题，同一条心电图曲线，可能出现多种心率失常的状态。
由于心电图的特性，使用类似时间序列的模型最佳

In [4]:
import os
import numpy as np
import pandas as pd

In [3]:
# constant
root_dir = r'C:/D/nsmc/TianChi/HeFei'
train_dir = os.path.join(root_dir, 'train')
testa_dir = os.path.join(root_dir, 'testA')
train_label_file = os.path.join(root_dir, 'hf_round1_label.txt')
arrythmia_file = os.path.join(root_dir, 'hf_round1_arrythmia.txt')

#### 加载train_data

In [ ]:
train_files = os.listdir(train_dir)
train_files.sort()
length = len(train_files)
train_data = np.zeros((length, 5000, 8))
for i, train_file in enumerate(train_files):
    data = np.loadtxt(one_train_file, skiprows=1)[np.newaxis, :]
    train_data[i, :] = data

#### 将心率失常的种类进行编码

In [71]:
with open(arrythmia_file, 'r') as fp:
    arrythmia = [i.strip() for i in fp.readlines()]
arrythmia_encoding = {}
arrythmia_decoding = {}
for i, key in enumerate(arrythmia):
    arrythmia_encoding[key] = i
    arrythmia_decoding[i] = key

#### 将标签进行one-hot编码

In [48]:
with open(train_label_file, 'r', encoding='utf-8') as fp:
    train_labels = [i.split('\t') for i in fp.readlines()]

In [60]:
print(len(train_labels))
print(train_labels[0:5])

24106
[['2.txt', '', '', '窦性心律', 'QRS低电压', '\n'], ['5.txt', '40', 'FEMALE', '窦性心律', 'QRS低电压', '\n'], ['9.txt', '54', 'FEMALE', '窦性心动过缓', 'QRS低电压', '\n'], ['12.txt', '25', 'FEMALE', '窦性心律', 'QRS低电压', '\n'], ['14.txt', '37', 'FEMALE', '窦性心律', 'QRS低电压', '\n']]


In [57]:
def to_one_hot(labels, dimension=55):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        for j in label[3:]:
            index = arrythmia_encoding.get(j)
            if index is not None:
                results[i, index] = 1.
    return results

In [61]:
one_hot_train_labels = to_one_hot(train_labels)

In [77]:
print(len(one_hot_train_labels))
print(one_hot_train_labels[0:5])

24106
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]]


In [76]:
print('种类\t数量')
for i, count in enumerate(np.sum(one_hot_train_labels, axis=0).astype(np.int)):
    print(arrythmia_decoding[i], count)

种类	数量
窦性心律 16918
T波改变 3421
窦性心动过缓 3372
ST段改变 2967
正常ECG 4171
左心室高电压 4326
ST-T改变 2111
窦性心动过速 2010
临界ECG 1911
QRS低电压 1543
房性早搏 1470
电轴左偏 1137
电轴右偏 1055
心房颤动 1217
异常ECG 1061
室性早搏 971
完全性右束支传导阻滞 1109
窦性心律不齐 924
左心室肥大 432
右束支传导阻滞 392
一度房室传导阻滞 282
快心室率 229
不完全性右束支传导阻滞 199
非特异性T波异常 125
QT间期延长 101
左前分支传导阻滞 106
非特异性ST段异常 78
差异性传导 75
非特异性ST段与T波异常 61
起搏心律 74
短PR间期 55
下壁异常Q波 53
快室率心房颤动 42
逆钟向转位 34
室内差异性传导 36
早期复极化 37
二联律 27
室上性早搏 33
顺钟向转位 30
复极化异常 29
未下传的房性早搏 25
肺心病型 27
慢心室率 30
短串房性心动过速 21
非特异性室内传导延迟 16
右心房扩大 24
左束支传导阻滞 18
前间壁R波递增不良 19
右心室肥大 18
房室传导延缓 10
双分支传导阻滞 20
非特异性室内传导阻滞 17
肺型P波 17
完全性左束支传导阻滞 20
融合波 18
